In [4]:
from pyspark import SparkContext

swan_spark_conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local")
swan_spark_conf \
  .set("spark.kubernetes.namespace", "spark-ml") \
  .set("spark.kubernetes.authenticate.executor.serviceAccountName", "spark-ml-executor") \
  .set("spark.executor.instances", "2") \
  .set("spark.executor.memory", "2g") \
  .set("spark.executor.cores", "2") \
  .set("spark.driver.memory", "1G") \
  .set("spark.kubernetes.container.image", "spark:3.0.1_python386") \
  .set("spark.driver.port", "2222") \
  .set("spark.driver.blockManager.port", "7777") \
  .set("spark.driver.host", "jupyter.recs.svc.cluster.local") \
  .set("spark.driver.bindAddress", "0.0.0.0") \
  .set("spark.ui.port", "4040") \
  .set("spark.network.timeout", "240")
print(swan_spark_conf.toDebugString()) #Instance of SparkConf with options set by the extension

sc = SparkContext.getOrCreate(conf=swan_spark_conf)


import time
b=sc.broadcast([3,5]) #Creating a broadcast variable available on all executors
a=sc.accumulator(0)   #Creating an accumulator for adding values across executors
RDD0=sc.parallelize([y for y in range(0,5)]) #RDD from input python collection
RDD2=sc.parallelize([z for z in range(10,15)])
RDD1=RDD0.cartesian(RDD2) 
cached=RDD2.cache() #Testing cached RDD
RDD22=RDD1.map(lambda x:x[0]+x[1]+b.value[0])
RDD3=RDD22.repartition(5) # To trigger a new stage.
RDD4=RDD2.map(lambda x: 3*x-b.value[0])
RDD5=RDD3.filter(lambda x:x%2==0)
RDD6=RDD4.filter(lambda x:x%2!=0)
RDD7=RDD5.cartesian(RDD6)
RDD8=RDD7.flatMap(lambda x: [x[i] for i in range(0,2)])
RDD9=RDD8.union(cached)
ans=RDD9.reduce(lambda x,y: x+y) # Doing a simple sum on the random data.
print(ans)
def f(x):
    global a
    time.sleep(0.5) #Making the job run a little longer
    a+=x
RDD9.foreach(f)
print(a.value)
#Display should appear automatically
sc.stop()

spark.extraListeners=sparkmonitor.listener.JupyterSparkMonitorListener
spark.driver.extraClassPath=/opt/conda/lib/python3.8/site-packages/sparkmonitor/listener_2.12.jar
spark.master=k8s://https://kubernetes.default.svc.cluster.local
spark.kubernetes.namespace=spark-ml
spark.kubernetes.authenticate.executor.serviceAccountName=spark-ml-executor
spark.executor.instances=2
spark.executor.memory=2g
spark.executor.cores=2
spark.driver.memory=1G
spark.kubernetes.container.image=spark:3.0.1_python386
spark.driver.port=2222
spark.driver.blockManager.port=7777
spark.driver.host=jupyter.recs.svc.cluster.local
spark.driver.bindAddress=0.0.0.0
spark.ui.port=4040
spark.network.timeout=240
spark.extraListeners=sparkmonitor.listener.JupyterSparkMonitorListener
spark.driver.extraClassPath=/opt/conda/lib/python3.8/site-packages/sparkmonitor/listener_2.12.jar
spark.master=k8s://https://kubernetes.default.svc.cluster.local
spark.kubernetes.namespace=spark-ml
spark.kubernetes.authenticate.executor.serviceA